In [ ]:
import gradio as gr
import numpy as np
import mongodb as db
import pandas as pd
import json
from datetime import datetime
import copy

In [ ]:
# check existing collections
collections = db.mydb.list_collection_names()
for collection in collections:
    print (collection)

In [ ]:
docs = db.MemberColl.find()
for doc in docs:
    print(doc)

In [ ]:
# query type = in, out as a status
query = {'type': {'$ne': 'reg'}}
docs = db.MemberColl.find(query)
for doc in docs:
    print(doc)

In [ ]:
time = db.datetime.now()
time = db.datetime.today()
print(time)

In [ ]:
query = {'type': {'$ne': 'reg'}}
#doc = db.MemberColl.find_one(query, sort=[('_id', -1)])\
doc = db.MemberColl.find_one(query, sort=[('date', -1)])
print(doc)

In [ ]:
for ele in doc:
    if ele == "_id": pass
    else: print(f"{ele}: {doc[ele]}")

In [ ]:
query = {'type': {'$ne': 'reg'}}
projection = {"_id": 0}         # 0 mean exclude _id from query result
docs = db.MemberColl.find(query, projection)
# for doc in docs:
#     print(doc)

# data_list = [{'ID': '111', 'name': 'ElonMusk', 'status': 1, 'type': 'in', 'date': '2023, 4, 20, 21, 37, 18, 716000'}, 
# {'ID': '111', 'name': 'ElonMusk', 'status': 0, 'type': 'out', 'date': '2023, 4, 20, 21, 37, 33, 267000'}]

data_list = list(docs)
# Handle datetime format -> iso format
for data in data_list:
    data['date'] = data['date'].isoformat()

#print(data_list)
data_json = json.dumps(data_list)

data = json.loads(data_json)
df = pd.DataFrame(data)
print(df)    

In [ ]:
print(df)

In [ ]:
def run_query():
    query = {'type': {'$ne': 'reg'}}
    projection = {"_id": 0}         # 0 mean exclude _id from query result
    docs = db.MemberColl.find(query, projection)

    data_list = list(docs)

    # Handle datetime format -> iso format
    for data in data_list:
        data['date'] = data['date'].isoformat()

    # convert to json type
    data_json = json.dumps(data_list)

    data = json.loads(data_json)
    df = pd.DataFrame(data)
    return df

print(data)
with gr.Blocks() as demo:
    with gr.Tab("Logs"):
        gr.Markdown("# 📝 Log Status Dashboard")
        btn = gr.Button("Show Log")
        btn.click(run_query, outputs=gr.DataFrame(run_query))
    
demo.launch()


In [ ]:
# Handle data
# convert datetime format to get a day
def getDay(date):
    date_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%f')
    day = date_obj.day
    return str(day)

data2 = copy.deepcopy(data)
    
for obj in data2:
    obj['date'] = getDay(obj['date'])

print(data2)
print(data)



In [ ]:
graph_data = [{'type': data2[0]['type'], 'date': data2[0]['date'], 'count': 0}]

for i in range(1,len(data2)):
    j = 0
    while (j < len(graph_data)):
        if data2[i]['date'] == graph_data[j]['date'] and data2[i]['type'] == graph_data[j]['type']:
            break
        j += 1
    if j != len(graph_data): continue
    else: graph_data.append({'type': data2[i]['type'], 'date': data2[i]['date'], 'count': 0})

for i in range(len(data2)):
    if data2[i]['type'] == 'in':
        for j in range(len(graph_data)):
            if graph_data[j]['type'] == 'in' and graph_data[j]['date'] == data2[i]['date']:
                graph_data[j]['count'] += 1
                break
        continue
    else:
        for j in range(len(graph_data)):
            if graph_data[j]['type'] == 'out' and graph_data[j]['date'] == data2[i]['date']:
                graph_data[j]['count'] += 1
                break
        continue

count = 0    
for i in range(len(graph_data)):
    count += graph_data[i]['count']


print(data2)
print(graph_data)
print(len(graph_data))
print(count)



In [ ]:
df_graph = pd.DataFrame(graph_data)

In [ ]:

def line_plot_fn():
    return gr.BarPlot.update(
            df_graph,
            x="date",
            y="count",
            color="type",
            #color_legend_position="bottom",
            title="Frequency",
            tooltip=['date', 'count', 'type'],
            height=500,
            width=500
    )

with gr.Blocks() as line_plot:
      plot = gr.BarPlot(show_label=False).style(container=False)
      #df.change(line_plot_fn, inputs=df, outputs=plot)
      line_plot.load(fn=line_plot_fn, outputs=plot)
        

line_plot.launch()